last edited on November 3 by Claire Valva
# Endless seasons for 40.5N spectral analysis

This notebook uses fft coefficient values (averaged for each season) to then create an arbitrarily long wave (ie. an endless summer etc) for later analysis. Currently known to work when the season wanted is a total of 10 by 10 observations (which is not quite endless). 

In [1]:
#use this instead? it might be faster? 
#maybe do numpy.reshape instead? after just running everything in line? download this later!
import numexpr as ne
from numba import jit

In [2]:
#import packages
import numpy as np
from scipy.fftpack import fft, ifft, fftfreq, fftshift, ifftshift
import matplotlib.pyplot as plt
from math import pi
import matplotlib.ticker as tck
import datetime
import pickle
from functions_forspectralanalysis import * #just in case I need it?

/home/clairev/python-climate-data-processing/submission_from_claire/40.5N-processing/functions_forspectralanalysis.py:84: RuntimeWarning: divide by zero encountered in true_divide
  zonal_spacing = 1/zonal_spacing


In [3]:
#load 2d transform from last time
file_Name = "transform_2d_oct2018"
file_pickle = open(file_Name,'rb') 
loaded_pickle = pickle.load(file_pickle)

In [4]:
d2_trans, seasons, d2_averages = loaded_pickle

## ifft data (with simulation of extended time)

In [5]:
#get some constants/often used arrays
M = len(d2_trans[0])
N = len(d2_trans[0][0])

exp_part = 2*pi*1j
divided = M*N
recip_divided = 1/(M*N)

m_array = [m for m in range(M) for n in range(N)]
n_array = [n for m in range(M) for n in range(N)]

In [6]:
#set i,k,l for testing purposes
i = 3
k = 4
l = 5

#get this array for testing
test_fft_array = np.array([d2_trans[i][m][n] 
                           for m in range(M) 
                           for n in range(N)])

to_exps = np.array(ne.evaluate("m_array*k/M+n_array*l/N")) * exp_part
exped = ne.evaluate("exp(to_exps)")

### time and test functions to make sure they run with numba

In [7]:
import time

#declare the test array <- im ok with this not being optimized because one has to do it 4 times
#if that changes, may have to optimize further
fft_array = test_fft_array

@jit(nopython = True) #try to do this with numba
def fast_g(k,l,fft_array):
    #function gets each individual coefficient
    to_exps = np.array([m*k/M+n*l/N for m in range(M) for n in range(N)]) * exp_part
    exped = np.exp(to_exps)
    entry_kl = np.sum(fft_array * exped)
    
    return entry_kl

@jit(nopython = True) 
def fast_bundled(fft_array = fft_array, K = 240, L = 2000):
    #function that gets all coefficients in a nice array, will be divided by M*N at end
    return [[fast_g(k,l,fft_array) for l in range(L)] for k in range(K)]


# time fast_g with compilation
start = time.time()
fast_g(1,2,fft_array)
end = time.time()
print("fast_g: elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
fast_g(1,2,fft_array)
end = time.time()
print("fast_g: elapsed (after compilation) = %s" % (end - start))

#time it?
start = time.time()
fast_bundled(K=100,L=10)
end = time.time()
print("fast_bundled: elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
fast_bundled(K=100,L=10)
end = time.time()
print("fast_bundled: elapsed (after compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
[[fast_g(k,l,fft_array) for k in range(100)] for l in range(10)]
end = time.time()
print("check if fast_bundled adds anything: (after compilation) = %s" % (end - start))

fast_g: elapsed (with compilation) = 0.7740650177001953
fast_g: elapsed (after compilation) = 0.008321046829223633
fast_bundled: elapsed (with compilation) = 8.430325746536255
fast_bundled: elapsed (after compilation) = 7.551801681518555
check if fast_bundled adds anything: (after compilation) = 7.4927544593811035


In [8]:
#define total result (by also multiplying by divided)
def total_ifft(fft_array, K, L):
    return np.array(fast_bundled(fft_array = fft_array,K = K,L = L))*recip_divided

### now apply functions to the seasonal averages (and a sample)

In [9]:
#get list of fft coeffs will apply to for later labeling
fft_names = seasons + ["test"]

#flatten averages for functions
flat_avgs = [[d2_averages[i][m][n] 
                           for m in range(M) 
                           for n in range(N)] for i in range(4)]

#get list of arrays to apply functions
toapply = flat_avgs.copy()
toapply.append(test_fft_array)

toapply = np.array(toapply)

In [ ]:
#apply ifft
verylongseasons = [total_ifft(fft_array = coeffs, K = 240, L = 2000) for coeffs in toapply]

In [ ]:
file_Name = "nov18"
    
file_pickle = open(file_Name,'wb') 
savethis = [verylongseasons, fft_names]   
pickle.dump(savethis,file_pickle)
file_pickle.close()